In [0]:
dbutils.widgets.dropdown("Process_name","process_name1",['process_name1','process_name2','process_name3'])
dbutils.widgets.text("s3_location","s3://tst-bkt/inbound/project_name/")

In [0]:
s3_location = dbutils.widgets.get("s3_location")
Process_name = dbutils.widgets.get("Process_name")

In [0]:
%run ./Parameters

In [0]:
object_list = object_name[Process_name]
print(object_list)

In [0]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import lit, struct
from functools import reduce
for table_name in object_list:
  #table_name = dbutils.widgets.get("table_name")
  print("table_name", table_name)
  s3_folder = f"{s3_location}/{Process_name}/{table_name}/delta"
  #print(s3_folder)
  folder_list = dbutils.fs.ls(s3_folder)
  #print(folder_list[0].path)
  folder_list.sort(reverse=True)
  latest_folder = folder_list[0].path
  #print(latest_folder)
  file_list = dbutils.fs.ls(latest_folder)
  #print(file_list)
  for file in file_list:
    print(file.path)
  df = spark.read.option("header", "true").csv(file.path, inferSchema=True, sep='|')
  for cols in df.columns:
    df = df.withColumnRenamed(cols, cols.replace('.', '_'))
  datatype = df.dtypes
  #print(datatype)
  df2 = spark.createDataFrame(datatype, ['Field_Name', 'Data_Type']).withColumn("File_name", lit(table_name))
  df2.createOrReplaceTempView("s3file_metadata")

  df3 = spark.sql(f"desc env_name.src_schema.{table_name}")
  src_table_name = "src_temp_table"
  df3.createOrReplaceTempView(src_table_name)

  df4 = spark.sql(f"desc env_name.pub_schema.{table_name}")  
  pub_table_name = "pub_temp_table"
  df4.createOrReplaceTempView(pub_table_name)

  df5 = spark.sql(f"""select s3file_metadata.Field_Name as s3_field_name, s3file_metadata.Data_Type as s3_datatype, src_temp_table.col_name src_col_name,       
                      src_temp_table.data_Type src_data_Type, pub_temp_table.col_name pub_col_name, pub_temp_table.data_Type src_data_Type,   
                      case when src_temp_table.col_name = s3file_metadata.Field_Name then 'true' else 'false' end as s3_src_col,
                      case when src_temp_table.col_name = pub_temp_table.col_name then 'true' else 'false' end as src_pub_col
                      from s3file_metadata full outer join src_temp_table
                      on trim(s3file_metadata.Field_Name)  = trim(src_temp_table.col_name)
                      full outer join pub_temp_table
                      on trim(src_temp_table.col_name)  = trim(pub_temp_table.col_name) order by s3_src_col desc"""
                  )
  df5.createOrReplaceTempView("s3_src_pub_table")
  df5.display()
